In [1]:
import sys

In [2]:
sys.path.append('/var/www/pipelines/celery_app')

In [3]:
from datetime import datetime

In [4]:
from tasks import get_stock_info

In [5]:
res = get_stock_info.delay('FB', datetime(2016, 1, 1), datetime.today())

In [6]:
res.status

'STARTED'

In [7]:
res.get()

'{"Close min":{"FB":94.160004},"Close max":{"FB":125.260002},"Close mean":{"FB":112.8609373937},"Close median":{"FB":113.9300005},"Open min":{"FB":92.830002},"Open max":{"FB":127.519997},"Open mean":{"FB":112.8902501312},"Open median":{"FB":113.8500025},"Adj Close min":{"FB":94.160004},"Adj Close max":{"FB":125.260002},"Adj Close mean":{"FB":112.8609373937},"Adj Close median":{"FB":113.9300005},"Low min":{"FB":89.370003},"Low max":{"FB":124.75},"Low mean":{"FB":111.5968122063},"Low median":{"FB":112.959999},"High min":{"FB":95.0},"High max":{"FB":128.330002},"High mean":{"FB":113.9396873375},"High median":{"FB":114.7299995}}'

In [8]:
res = get_stock_info.apply_async(('FB', datetime(2016, 1, 1), datetime.today()), queue='priority')

In [9]:
res.get()

'{"Open min":{"FB":92.830002},"Open max":{"FB":127.519997},"Open mean":{"FB":112.8902501312},"Open median":{"FB":113.8500025},"Close min":{"FB":94.160004},"Close max":{"FB":125.260002},"Close mean":{"FB":112.8609373937},"Close median":{"FB":113.9300005},"Low min":{"FB":89.370003},"Low max":{"FB":124.75},"Low mean":{"FB":111.5968122063},"Low median":{"FB":112.959999},"Adj Close min":{"FB":94.160004},"Adj Close max":{"FB":125.260002},"Adj Close mean":{"FB":112.8609373937},"Adj Close median":{"FB":113.9300005},"High min":{"FB":95.0},"High max":{"FB":128.330002},"High mean":{"FB":113.9396873375},"High median":{"FB":114.7299995}}'

In [10]:
from tasks import get_stock_info, price_range, determine_buy, sort_results

In [11]:
from celery import chain, group, chord

In [12]:
start = datetime(2016, 1, 1)

In [13]:
end = datetime.today()

In [14]:
my_chain = chain(price_range.s('FB', start, end), determine_buy.s())

In [15]:
my_chain

tasks.price_range('FB', datetime.datetime(2016, 1, 1, 0, 0), datetime.datetime(2016, 8, 22, 13, 26, 38, 292856)) | tasks.determine_buy()

In [16]:
res = my_chain()

In [17]:
res

<AsyncResult: 83770b43-3ae1-4a85-9066-3426ee482b94>

In [18]:
res.status

'SUCCESS'

In [19]:
res.get()

False

In [20]:
my_stocks = ['FB', 'GOOG', 'IBM']

In [21]:
my_group = group(get_stock_info.s(stk, start, end) for stk in my_stocks)

In [22]:
my_group

[tasks.get_stock_info('FB', datetime.datetime(2016, 1, 1, 0, 0), datetime.datetime(2016, 8, 22, 13, 26, 38, 292856)), tasks.get_stock_info('GOOG', datetime.datetime(2016, 1, 1, 0, 0), datetime.datetime(2016, 8, 22, 13, 26, 38, 292856)), tasks.get_stock_info('IBM', datetime.datetime(2016, 1, 1, 0, 0), datetime.datetime(2016, 8, 22, 13, 26, 38, 292856))]

In [23]:
res = my_group()

In [24]:
res

<GroupResult: ad8c46f7-4032-4028-a563-c44391ab400e [7089d1ed-9335-4002-9de4-544e4b899cff, 5840e896-9b8e-48ff-b6fa-a67ad1ca9212, 4762aa4c-c1ae-475d-94af-6afbe7a3ae66]>

In [25]:
res.get()

['{"Low min":{"FB":89.370003},"Low max":{"FB":124.75},"Low mean":{"FB":111.5968122063},"Low median":{"FB":112.959999},"High min":{"FB":95.0},"High max":{"FB":128.330002},"High mean":{"FB":113.9396873375},"High median":{"FB":114.7299995},"Open min":{"FB":92.830002},"Open max":{"FB":127.519997},"Open mean":{"FB":112.8902501312},"Open median":{"FB":113.8500025},"Close min":{"FB":94.160004},"Close max":{"FB":125.260002},"Close mean":{"FB":112.8609373937},"Close median":{"FB":113.9300005},"Adj Close min":{"FB":94.160004},"Adj Close max":{"FB":125.260002},"Adj Close mean":{"FB":112.8609373937},"Adj Close median":{"FB":113.9300005}}',
 '{"Open min":{"GOOG":667.849976},"Open max":{"GOOG":785.0},"Open mean":{"GOOG":724.2199360063},"Open median":{"GOOG":722.4649965},"Close min":{"GOOG":668.26001},"Close max":{"GOOG":784.849976},"Close mean":{"GOOG":724.18143845},"Close median":{"GOOG":719.6299745},"Low min":{"GOOG":663.059998},"Low max":{"GOOG":782.969971},"Low mean":{"GOOG":717.8580696687},"Low

In [26]:
header = (price_range.subtask((stk, start, end)) for stk in my_stocks)

In [27]:
callback = sort_results.subtask()

In [28]:
%timeit chord(header)(callback).get()

1 loop, best of 3: 1.08 s per loop


In [29]:
%timeit sort_results(group(header)().get())

The slowest run took 34.82 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 67.9 µs per loop
